In [19]:
from pprint import pprint
import requests
from bs4 import BeautifulSoup

url_list = [
    'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=1',
    'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=2'
]

result = requests.get(url_list[0])

if result.status_code == 200:
    soup = BeautifulSoup(result.content, 'html.parser')

    products = soup.select('div.items__item.item-card')
    # pprint(products)

    items = []

    for adv in products:
        data = {
            'name': adv.select('h2[class*=ItemTitle-module_item-title__]'),
            # 'name': adv.select('h2[class*=ItemTitle-module_item-title__]')[0] \
            #     .get_text() if adv.select('h2[class*=ItemTitle-module_item-title__]') else None,
            'price': adv.select('p.price'),
            'town': adv.select('span[class*=index-module_town__]'),
            'city': adv.select('span.city'),
            'shipping': adv.select('span.shipping-badge'),
            'sold': adv.select('span.item-sold-badge'),  
        }

        for key, value in data.items():
            if value:
                data[key] = value[0].get_text()
            else:
                data[key] = None

            # data[key] = value[0].get_text() if value else None

        data.update({
            'price': float(data['price']
                    .replace('Spedizione disponibile', '')
                    .replace('\xa0€', '')
                    .replace('Venduto', '')),
            'url': adv.select('a.link')[0]['href'],
        })

        # [0].get_text().replace('Spedizione disponibile', '').replace('Venduto', ''),
        
        items.append(data)
    
    pprint(items)

{'city': '(PV)',
 'name': 'Zx spectrum+2 128k sinclair',
 'price': 300.0,
 'shipping': 'Spedizione disponibile',
 'sold': None,
 'town': 'Pavia ',
 'url': 'https://www.subito.it/informatica/zx-spectrum-2-128k-sinclair-pavia-494925569.htm'}
{'city': '(VE)',
 'name': 'ZX Spectrum SINCLAIR',
 'price': 220.0,
 'shipping': 'Spedizione disponibile',
 'sold': None,
 'town': 'Noventa di Piave ',
 'url': 'https://www.subito.it/videogiochi/zx-spectrum-sinclair-venezia-494813174.htm'}
{'city': '(TO)',
 'name': 'Sinclair ZX Spectrum 16kb',
 'price': 550.0,
 'shipping': 'Spedizione disponibile',
 'sold': None,
 'town': 'Torino ',
 'url': 'https://www.subito.it/informatica/sinclair-zx-spectrum-16kb-torino-494749976.htm'}
{'city': '(SA)',
 'name': 'Disco 2tb Batocera v36',
 'price': 100.0,
 'shipping': 'Spedizione disponibile',
 'sold': None,
 'town': 'Pagani ',
 'url': 'https://www.subito.it/informatica/disco-2tb-batocera-v36-salerno-494644761.htm'}
{'city': '(SA)',
 'name': 'Disco 2tb Batocera',
 '

In [1]:
from pprint import pprint
from time import sleep
import requests
from bs4 import BeautifulSoup
from gestione_input import leggi_float

# url_list = [
#     'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=1',
#     'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=2'
# ]

base_url = 'https://www.subito.it/annunci-italia/vendita/usato/'


def scrape_page(page):
    query_str = {
        'q': 'spectrum+sinclair',
        'o': page
    }

    result = requests.get(base_url, params=query_str)

    if result.status_code == 200:
        soup = BeautifulSoup(result.content, 'html.parser')

        # Controllare se non è una pagina vuota
        no_results = soup.select('div[class*=ErrorLayout_container__]')
        if no_results:
            raise requests.RequestException('Non ci sono più pagine da analizzare.')
        
        products = soup.select('div.items__item.item-card')
        # pprint(products)

        items = []

        for adv in products:
            data = {
                'name': adv.select('h2[class*=ItemTitle-module_item-title__]'),
                # 'name': adv.select('h2[class*=ItemTitle-module_item-title__]')[0] \
                #     .get_text() if adv.select('h2[class*=ItemTitle-module_item-title__]') else None,
                'price': adv.select('p.price'),
                'town': adv.select('span[class*=index-module_town__]'),
                'city': adv.select('span.city'),
                'shipping': adv.select('span.shipping-badge'),
                'sold': adv.select('span.item-sold-badge'),  
            }

            for key, value in data.items():
                if value:
                    data[key] = value[0].get_text()
                else:
                    data[key] = None

                # data[key] = value[0].get_text() if value else None

            data.update({
                'price': float(data['price']
                        .replace('Spedizione disponibile', '')
                        .replace('\xa0€', '')
                        .replace('Venduto', '')) if data['price'] else None,
                'url': adv.select('a.link')[0]['href'],
            })

            # [0].get_text().replace('Spedizione disponibile', '').replace('Venduto', ''),
            
            items.append(data)
        
    return items


curr_page = 1
all_data = []

max_page = leggi_float('Quante pagine vuoi analizzare? '
    'Premi semplicemente invio per cercare su tutte le pagine disponibili.',
    None, None)

while True:
    if not max_page or curr_page <= max_page:
    # if max_page is None or curr_page <= max_page:
        try:
            items = scrape_page(curr_page)
        except requests.RequestException as err:
            print(err)
            break
        
        all_data += items
        curr_page += 1
        sleep(1)
    else:
        break
    

pprint(all_data)


Non ci sono più pagine da analizzare.
[{'city': '(VR)',
  'name': 'Sinclair ZX Spectrum 48k personal computer',
  'price': 750.0,
  'shipping': 'Spedizione disponibile',
  'sold': None,
  'town': 'Nogarole Rocca ',
  'url': 'https://www.subito.it/informatica/sinclair-zx-spectrum-48k-personal-computer-verona-495121453.htm'},
 {'city': '(PV)',
  'name': 'Zx spectrum+2 128k sinclair',
  'price': 300.0,
  'shipping': 'Spedizione disponibile',
  'sold': None,
  'town': 'Pavia ',
  'url': 'https://www.subito.it/informatica/zx-spectrum-2-128k-sinclair-pavia-494925569.htm'},
 {'city': '(VE)',
  'name': 'ZX Spectrum SINCLAIR',
  'price': 220.0,
  'shipping': 'Spedizione disponibile',
  'sold': None,
  'town': 'Noventa di Piave ',
  'url': 'https://www.subito.it/videogiochi/zx-spectrum-sinclair-venezia-494813174.htm'},
 {'city': '(TO)',
  'name': 'Sinclair ZX Spectrum 16kb',
  'price': 550.0,
  'shipping': 'Spedizione disponibile',
  'sold': None,
  'town': 'Torino ',
  'url': 'https://www.subit

In [ ]:
from pprint import pprint
import requests
from bs4 import BeautifulSoup


# Lista di URL da esaminare
url_list = [
    'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=1',
    'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=2'
]

# Effettua una richiesta GET al primo URL nella lista
result = requests.get(url_list[0])

# Controlla se la richiesta ha avuto successo (status code 200)
if result.status_code == 200:
    # Parsing del contenuto HTML della pagina utilizzando BeautifulSoup
    soup = BeautifulSoup(result.content, 'html.parser')

    # Seleziona gli elementi HTML corrispondenti ai prodotti desiderati
    products = soup.select('div.items__item.item-card')
    # pprint(products)  # Stampa la lista di prodotti (opzionale)

# Lista per memorizzare gli oggetti di interesse
items = []

# Itera sui prodotti trovati
for adv in products:
    # Estrae i dati di interesse dal prodotto corrente
    data = {
        'name': adv.select('h2[class*=ItemTitle-module_item-title__]'),  # Nome del prodotto
        'price': adv.select("p.price"),  # Prezzo del prodotto
        'town': adv.select('span[class*=index-module_town__]'),  # Città del venditore
    }

    # Elabora i dati estratti
    for key, value in data.items():
        if value:
            # Se l'elemento è presente, estrai il testo
            data[key] = value[0].get_text()
        else:
            # Se l'elemento non è presente, imposta il valore a None
            data[key] = None

    # Stampa i dati estratti
    pprint(data)